# Fused Softmax

在本教程中，你将编写一个融合 Softmax 运算，该运算速度明显快于 PyTorch 针对特定矩阵类别的原生运算：即那些行可以放入 GPU 的 SRAM 中的矩阵。

在此过程中，你将了解：

* kernel fusion对于带宽受限运算的优势。

* Triton 中的约简运算符。

## 动机

用于元素加法的自定义 GPU 内核具有教育意义，但在实践中不会有太大帮助。
我们来考虑一个简单的，原始的（数值稳定的）softmax 运算：

In [1]:
import torch

import triton
import triton.language as tl
from triton.runtime import driver

# 选择当前处于“激活状态”的 PyTorch 设备（通常是 GPU 上的 'cuda' 或 'hip'）
DEVICE = triton.runtime.driver.active.get_active_torch_device()


def is_hip():
    # 当 Triton 当前目标后端为 'hip' 时返回 True。
    # 'hip' 指 AMD 的 ROCm/HIP 平台（与 NVIDIA 的 'cuda' 类似）。
    return triton.runtime.driver.active.get_current_target().backend == "hip"


def is_cdna():
    # 仅在 AMD 的 HIP 平台上，并且架构字符串属于 CDNA 系列（数据中心/计算型架构）时返回 True。
    # 这里的 'gfx940'/'gfx941'/'gfx942'（MI300 家族）以及 'gfx90a'、'gfx908'（早期 Instinct 系列）
    # 都是 CDNA 代际的标识；不同于面向游戏图形的 RDNA。
    return is_hip() and triton.runtime.driver.active.get_current_target().arch in ('gfx940', 'gfx941', 'gfx942',
                                                                                   'gfx90a', 'gfx908')


def naive_softmax(x):
    """使用原生 PyTorch 逐行（row-wise）计算 X 的 softmax。

    为避免溢出，我们会先减去每一行的最大值。softmax 对这种平移是不变的。
    """
    # 读取 MN 个元素；写入 M 个元素（每一行的最大值）
    x_max = x.max(dim=1)[0]
    # 读取 MN + M 个元素；写入 MN 个元素（将每行最大值广播后相减）
    z = x - x_max[:, None]
    # 读取 MN 个元素；写入 MN 个元素（逐元素指数）
    numerator = torch.exp(z)
    # 读取 MN 个元素；写入 M 个元素（每行求和，得到分母）
    denominator = numerator.sum(dim=1)
    # 读取 MN + M 个元素；写入 MN 个元素（逐元素相除，得到 softmax）
    ret = numerator / denominator[:, None]
    # 总计：读取 5MN + 2M 个元素；写入 3MN + 2M 个元素
    return ret


在 PyTorch 中简单实现时，计算 $x \in R^{M \times N}$ 的 `y = naive_softmax(x)` 需要从 DRAM 读取 $5MN + 2M$ 个元素，并写回 $3MN + 2M$ 个元素。
这显然很浪费；

我们更倾向于使用自定义的“fused”内核，它只读取一次 X 并在芯片上完成所有必要的计算。


## 计算内核（Compute Kernel）

我们的fused softmax内核的工作原理如下：

**并行策略**：
- 如vadd教程中那样，我们启动多个**并行程序**（parallel programs），每个程序负责处理输入矩阵X的一部分行
- 不过这次，各程序之间加上了**跨步访问**（strided access）模式，避免重复处理
- 每个程序将分配到的行加载到GPU的SRAM中，就地完成归一化计算，然后写回结果到输出矩阵Y

**关键概念**：
- `num_programs(0)`：返回在第0维度（行维度）上启动的并行程序总数
- `program_id(0)`：当前程序在第0维度上的唯一标识符（从0开始）
- **跨步访问**：程序i处理第i、i+num_programs、i+2×num_programs...行，确保所有行都被且仅被处理一次

让我们详细分析内核的实现：

**重要限制**：Triton要求每个块（block）必须包含**2的幂次个元素**。因此我们需要在内部对每行进行"填充"（padding），并通过掩码（mask）正确保护内存操作，以处理任意的输入形状。

In [2]:
@triton.jit
def softmax_kernel(
    output_ptr,              # 输出矩阵的首地址（指向第 0 行第 0 列）
    input_ptr,               # 输入矩阵的首地址（指向第 0 行第 0 列）
    input_row_stride,        # 输入矩阵“行步长”（单位是“元素个数”，通常等于列数；非字节！）
    output_row_stride,       # 输出矩阵“行步长”（同上，单位是元素）
    n_rows,                  # 矩阵的行数 M
    n_cols,                  # 矩阵的列数 N
    BLOCK_SIZE: tl.constexpr,# 每个程序（program）一次性在寄存器/SRAM里处理的“列块”大小
    num_stages: tl.constexpr # 供 Triton 做流水/预取的 hint，影响 for 循环展开/软件流水
):
    # ===== 任务划分（persistent/pipelined pattern）=====
    # 如教程1 vadd中已经讨论过的，Triton 的 kernel 不是只运行一次，而是会并行地启动多个“程序（program）”。
    # program_id(0) 给出当前这个程序在 axis=0（我们用来划分“行”）上的编号 pid。这里不再赘述。
    row_start = tl.program_id(0)

    # num_programs(0) 是 axis=0 上“实际启动的程序（program）总数”，由稍后的 host 侧的 num_programs 决定。
    # 这些程序将以“跨步”的方式分配行，形成一个等差序列：
    # 例如：n_rows=10、num_programs(0)=3，则
    #   pid=0 处理行 0,3,6,9
    #   pid=1 处理行 1,4,7
    #   pid=2 处理行 2,5,8
    # 这种以 num_programs 为步长的访问（i += row_step）称为“跨步并行/持久化线程（strided/persistent）”。
    #
    # 对比 vadd 教程：
    #   vadd 常用“大 grid”：grid = (ceil_div(n_elements, BLOCK_SIZE),)
    #   这里的“较大的”grid“数量接近于任务量，大grid的意思就是多个program。
    #   每个 program 只处理一个小块，处理完了程序也就跑完了，没有下一个任务，不在核内循环。
    #   如果照搬到softmax，n_rows=1e6 时，就要启动 1e6 个 program。那调度就是个灾难。
    #
    # 对于 softmax（逐行操作、单行工作量小）：
    #   我们选择“较小的 grid”（并发度 G，通常接近于硬件资源量，比如说处理器数量），小grid的意思是program少。
    #   这样无论 n_rows 是 1000 还是 1e9，都用同样的 grid 大小
    #   在核内用跨步循环让每个 program 处理多行，每次处理一行，减少过多 program 带来的启动/调度开销，并提升占用率。
    row_step = tl.num_programs(0)


    # ===== 逐行处理（但每个程序处理一个“等差序列”的行集合）=====
    # 如上方已经讨论的，与“每行一个 program”的做法不同，这里采用“持久化线程（persistent）”模式：
    #   - 好处：把“启动多少个 program”的选择与“有多少行要处理”解耦，
    #           便于抢占/负载均衡，以及控制并发度贴合硬件（SM/CU/波前）资源。
    #   - 这也是为什么我们需要 num_programs(0)：用于决定“跨步”的步长。
    for row_idx in tl.range(row_start, n_rows, row_step, num_stages=num_stages):
        # ==== 计算当前行的“行首指针” ====
        # 注意：*stride 是“以元素为单位”的距离*（PyTorch 的 stride 也是元素步长）
        # 若输入是行连续（C-contiguous），input_row_stride 通常等于 n_cols。
        row_start_ptr = input_ptr + row_idx * input_row_stride

        # ==== 构造一段“列索引向量”====
        # 我们将要一次性在 SRAM/寄存器里处理 BLOCK_SIZE 个列元素
        # BLOCK_SIZE 一般取 n_cols 的“上取最接近的 2 的幂的值”（见下文问答 #3）
        col_offsets = tl.arange(0, BLOCK_SIZE)

        # 这得到当前行里，从第 0 列开始、连续 BLOCK_SIZE 个位置的地址
        # 指针算术这里同样是“以元素为单位”的偏移
        input_ptrs = row_start_ptr + col_offsets

        # ==== 处理“边界/填充”的 mask ====
        # 若 n_cols 不是 2 的幂，BLOCK_SIZE 会比 n_cols 大（有“尾巴”）。而悲催的是，n_cols 通常不是完美的 2 的幂。
        # mask 确保只在有效列(0..n_cols-1)上读写，尾部越界列全部“屏蔽”。
        mask = col_offsets < n_cols

        # ==== 从 DRAM 把一整行（或带尾部填充的一段）搬到 SRAM ====
        # 对无效位置，用 -inf 填充，便于后续 max/sum 不被虚假值干扰（稳定 softmax 惯用法）
        row = tl.load(input_ptrs, mask=mask, other=-float('inf'))

        # ====== 融合 softmax（全在 SRAM/寄存器里算）======
        # 1) 减去最大值（数值稳定）
        row_minus_max = row - tl.max(row, axis=0)

        # 2) 指数（Triton 的 exp 是快速近似版，类似 CUDA 的 __expf）
        numerator = tl.exp(row_minus_max)

        # 3) 求和作为分母
        denominator = tl.sum(numerator, axis=0)

        # 4) 归一化
        softmax_output = numerator / denominator

        # ===== 显式地写回 DRAM =====
        # 计算输出矩阵这一行的“行首指针”
        output_row_start_ptr = output_ptr + row_idx * output_row_stride

        # 再加上“列偏移向量”得到这一行里要写回的所有元素地址
        output_ptrs = output_row_start_ptr + col_offsets

        # 仅在有效列上写回，潜在的尾部越界的 lane 通过 mask 屏蔽
        tl.store(output_ptrs, softmax_output, mask=mask)

现在我们可以创建一个host侧函数，为任意给定的输入张量将内核及其（元）参数加入执行队列。

该函数的关键作用包括：
1. **自动计算最优参数**：根据输入矩阵形状确定BLOCK_SIZE、num_warps等参数
2. **占用率优化**：基于GPU硬件特性计算最优的并行程序数量
3. **内核启动**：将优化后的参数传递给softmax_kernel并启动执行



In [3]:
# ===== GPU硬件信息获取 =====
# 这一步是为了了解我们运行的GPU硬件规格，以便后续优化并行度和资源分配
properties = driver.active.utils.get_device_properties(DEVICE.index)

# NUM_SM: Streaming Multiprocessors（流多处理器）的数量，这是GPU的"核心处理单元"数量
# 可以理解为CPU的"核数"，但GPU的SM比CPU核心简单得多，数量也多得多
# 例如RTX 4090有128个SM，A100有108个SM
NUM_SM = properties["multiprocessor_count"]

# NUM_REGS: 每个SM可用的寄存器总数，寄存器是GPU上最快的存储
# 寄存器资源有限且珍贵，如果一个kernel用太多寄存器，SM能并行运行的线程数就会减少
NUM_REGS = properties["max_num_regs"]

# SIZE_SMEM: 每个SM的共享内存（Shared Memory）大小，速度比DRAM快很多倍，但比寄存器慢
# 我们上面提到的SRAM实际上指的就是这个共享内存区域
SIZE_SMEM = properties["max_shared_mem"]

# WARP_SIZE: 一个warp（线程束）包含的线程数，NVIDIA GPU通常是32，AMD通常是64
# GPU以warp为单位调度线程，同一warp内的线程必须执行相同的指令（SIMT模型）
WARP_SIZE = properties["warpSize"]

target = triton.runtime.driver.active.get_current_target()
kernels = {}  # 用于缓存编译好的kernel，避免重复编译


def softmax(x):
    """
    这个函数是softmax_kernel的"包装器"或"启动器"，负责：
    1. 根据输入数据自动计算最优的kernel参数
    2. 分析GPU硬件资源，确定最佳并行度
    3. 启动kernel执行
    """
    n_rows, n_cols = x.shape

    # ===== 计算BLOCK_SIZE =====
    # Triton要求block大小必须是2的幂次，这里找到大于等于n_cols的最小2的幂次
    # 例如：n_cols=1000时，BLOCK_SIZE=1024；n_cols=500时，BLOCK_SIZE=512
    # 为什么要这样？因为GPU的向量化指令和内存对齐都偏好2的幂次大小
    BLOCK_SIZE = triton.next_power_of_2(n_cols)

    # ===== 设置warp数量 =====
    # num_warps决定每个"program"用多少个warp来处理一行数据
    # 更多warp意味着更多并行性，但也意味着更多资源消耗
    # 这里用经验值8，在后续教程中会学到自动调优的方法
    num_warps = 8

    # ===== 软件流水线阶段数 =====
    # num_stages控制Triton编译器的软件流水线优化程度
    # 更多stage能隐藏内存延迟，但需要更多共享内存
    # 根据GPU的SMEM大小动态选择：大内存GPU用4阶段，小内存GPU用2阶段
    num_stages = 4 if SIZE_SMEM > 200000 else 2

    # ===== 分配输出张量 =====
    y = torch.empty_like(x)  # 创建与输入x相同形状和类型的空张量

    # ===== KERNEL预编译与性能分析 =====
    # 这是关键步骤！为什么要warmup？
    # 1. Triton采用JIT编译，第一次调用时需要编译kernel为GPU机器码
    # 2. warmup让我们提前编译，并获取关键的性能指标（寄存器使用量、共享内存使用量等）
    # 3. 有了这些指标，我们就能计算出最优的并行度（occupancy）
    kernel = softmax_kernel.warmup(
        y, x, x.stride(0), y.stride(0), n_rows, n_cols, 
        BLOCK_SIZE=BLOCK_SIZE, num_stages=num_stages, num_warps=num_warps, 
        grid=(1, )  # 这里的grid=(1,)是占位符，真正的grid大小稍后确定
    )
    
    # 初始化kernel的内部句柄，让我们能访问编译后的元数据
    kernel._init_handles()
    
    # 获取编译后kernel的资源使用情况
    n_regs = kernel.n_regs                # 每个线程使用的寄存器数量
    size_smem = kernel.metadata.shared    # 每个block使用的共享内存大小

    # ===== 占用率计算（性能优化的关键）=====
    # occupancy（占用率）= 实际运行的线程数 / GPU能支持的最大线程数
    # 高占用率通常意味着更好的性能，因为能更好地隐藏内存延迟
    
    if is_hip():  # AMD GPU的情况
        # NUM_REGS表示常规用途寄存器数。在CDNA架构上这是总寄存器的一半
        # 但并非总是如此，大多数情况下所有寄存器都能作为常规用途寄存器使用
        # ISA手册第3.6.4节（针对CDNA3）说明：
        # VGPR分为两个池：常规VGPR和累加VGPR。累加VGPR用于矩阵VALU指令，
        # 也可直接从内存加载。一个wave最多512个VGPR，每种类型256个。
        # 当wave的VGPR少于512个时，两种类型的数量是灵活的，不要求相等。
        NUM_GPRS = NUM_REGS
        if is_cdna():
            NUM_GPRS = NUM_REGS * 2

        # MAX_NUM_THREADS表示每个多处理器的最大常驻线程数
        # 除以WARP_SIZE得到每个CU（多处理器）能并行执行的最大wave数
        MAX_NUM_THREADS = properties["max_threads_per_sm"]
        max_num_waves = MAX_NUM_THREADS // WARP_SIZE
        
        # 计算寄存器限制的occupancy：考虑每个线程的寄存器使用量
        occupancy = min(NUM_GPRS // WARP_SIZE // n_regs, max_num_waves) // num_warps
    else:  # NVIDIA GPU的情况
        # 更简单的计算公式：总寄存器数 / (每线程寄存器数 × warp大小 × warp数量)
        occupancy = NUM_REGS // (n_regs * WARP_SIZE * num_warps)
    
    # 考虑共享内存的限制：如果共享内存不够，也会限制occupancy
    occupancy = min(occupancy, SIZE_SMEM // size_smem)
    
    # ===== 计算最优并行程序数 =====
    # num_programs = 每个SM的occupancy × SM总数
    # 这就是我们要启动的"程序"总数，每个程序会处理若干行数据
    num_programs = NUM_SM * occupancy

    # 但是，程序数不能超过行数，否则有些程序没事可做
    num_programs = min(num_programs, n_rows)

    # ===== 真正启动kernel执行 =====
    # 这里的kernel实际上是warmup返回的"已编译kernel对象"，而不是原始的softmax_kernel函数
    # kernel[(num_programs, 1, 1)]的语法是Triton的特色：
    #   - (num_programs, 1, 1) 指定3D grid的大小，这里只用第一维
    #   - 这相当于CUDA的 <<<grid_size, block_size>>> 语法
    #   - 每个grid位置会启动一个"程序"，总共启动num_programs个程序
    kernel[(num_programs, 1, 1)](
        y, x, x.stride(0), y.stride(0), n_rows, n_cols, BLOCK_SIZE, num_stages
    )
    return y

## 基准测试

我们确保在行数和列数不规则的矩阵上测试我们的内核。
这将使我们能够验证我们的填充机制是否有效。

In [4]:
torch.manual_seed(0)
x = torch.randn(1823, 781, device=DEVICE)
y_triton = softmax(x)
y_torch = torch.softmax(x, axis=1)
assert torch.allclose(y_triton, y_torch), (y_triton, y_torch)

正如预期的那样，结果是相同的。



## Benchmark

我们将根据输入矩阵的列数（假设为 4096 行）对我们的操作进行基准测试。
然后，我们将其性能与 (1) `torch.softmax` 和 (2) 上面定义的 `naive_softmax` 进行比较。


In [ ]:
@triton.testing.perf_report(  # 装饰器：把下面的 benchmark() 函数“注册”为一组可自动跑、自动绘图/汇总的基准测试
    triton.testing.Benchmark(
        x_names=['N'],                       # 横轴使用的自变量名字（这里横轴是列数 N）
        x_vals=[128 * i for i in range(2, 100)],  # 横轴取值列表：从 256 到 128*99，步长 128
        line_arg='provider',                 # 折线的区分维度（相当于 legend 的类别）
        line_vals=['triton', 'torch', 'naive_softmax'],  # 折线对应的取值（3 条线）
        line_names=["Triton", "Torch", "Naive Softmax"], # 图例显示的名字
        styles=[('blue', '-'), ('green', '-'), ('red', '-')],  # 每条线的样式（颜色、线型）
        ylabel="GB/s",                       # 纵轴标题（单位：GiB/s 的近似，严格说是 10^9 byte/s）
        plot_name="softmax-performance",     # 图的名字（也会作为保存文件名的一部分）
        args={'M': 4096},                    # 固定参数：这里把行数 M 固定为 4096，横轴只扫 N
        # 也可以在这里加 quantiles / rep / warmup 等参数（见下文说明）
    ))
def benchmark(M, N, provider):
    # 为每个 (M, N, provider) 组合，构造输入张量并测一次速度
    x = torch.randn(M, N, device=DEVICE, dtype=torch.float32)

    # do_bench 期望接收一个“可调用对象”，它会自己负责多次调用以统计耗时
    # 传 lambda 的目的是“延迟执行”，把真正的调用权交给 do_bench（而不是我们在这里直接调用）
    if provider == 'torch':
        ms = triton.testing.do_bench(lambda: torch.softmax(x, dim=-1))
    if provider == 'triton':
        ms = triton.testing.do_bench(lambda: softmax(x))
    if provider == 'naive_softmax':
        ms = triton.testing.do_bench(lambda: naive_softmax(x))

    # 把时间（毫秒）换算成“有效带宽”GB/s：
    # 公式：带宽 ≈ 传输字节数 / 时间
    # 朴素估算：softmax 至少要“读一次 + 写一次” -> 2 * x.numel() * 每元素字节数
    # 注意：真实内核可能读写更多/更少，这里用于一致的粗略对比
    gbps = lambda ms: 2 * x.numel() * x.element_size() * 1e-9 / (ms * 1e-3)
    return gbps(ms)


# 运行基准：show_plots=True 会在本地弹出/保存图；print_data=True 会打印出表格数据
benchmark.run(show_plots=True, print_data=True)


在上图中，我们可以看到：
- Triton 比 Torch JIT 快 4 倍。这证实了我们的猜测，即 Torch JIT 在此处没有进行任何融合。
- Triton 明显比 `torch.softmax` 快——而且**更易于阅读、理解和维护**。
但请注意，PyTorch 的 `softmax` 运算更通用，可以处理任何形状的张量。
